In [2]:
from helpers import *
import pandas as pd
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel
import torch
import time
import datetime 

In [3]:
train_dataset = "./datas/data_train.csv"
test_dataset = "./datas/sampleSubmission.csv"

In [4]:
train = load_dataset(train_dataset)
test = load_dataset(test_dataset)
# train_tr, test_tr = split_dataset(train_df)

In [27]:
# Explicitly convert into datatypes needed by spotlight models
user_tr = np.array(train.User, dtype=np.int32)
movie_tr = np.array(train.Movie, dtype=np.int32)
rating_tr = np.array(train.Rating, dtype=np.float32)
user_te = np.array(test.User, dtype=np.int32)
movie_te = np.array(test.Movie, dtype=np.int32)
# num_users = len(train.User.unique())
# num_items = len(train.Movie.unique())

In [6]:
#spotlight.factorization.explicit.ExplicitFactorizationModel(loss='regression', embedding_dim=32, n_iter=10, 
#batch_size=256, l2=0.0, learning_rate=0.01, optimizer_func=None, use_cuda=False, representation=None, 
#sparse=False, random_state=None)

#(user_ids, item_ids, ratings=None, timestamps=None, weights=None, num_users=None, num_items=None)

train_data = Interactions(user_ids=user_tr, item_ids=movie_tr, ratings=rating_tr)
test_data = Interactions(user_ids=user_tr, item_ids=movie_te)

In [6]:
model = ExplicitFactorizationModel(loss='regression',
                                   embedding_dim=128,  # latent dimensionality
                                   n_iter=10,  # number of epochs of training
                                   batch_size=1024,  # minibatch size
                                   l2=1e-9,  # strength of L2 regularization
                                   learning_rate=1e-3,
                                   use_cuda=torch.cuda.is_available())

In [7]:
model.fit(train_data, verbose=True)

Epoch 0: loss 3.7733740256661954
Epoch 1: loss 1.0350730077080104
Epoch 2: loss 1.0120432836594788
Epoch 3: loss 0.9867667126655578
Epoch 4: loss 0.9654490405061971
Epoch 5: loss 0.9378023430575495
Epoch 6: loss 0.8966138672828674
Epoch 7: loss 0.8377001707968504
Epoch 8: loss 0.7613696554951046
Epoch 9: loss 0.6713688180757605


In [8]:
# train_rmse = rmse_score(model, train_tr_data)
# test_rmse = rmse_score(model, test_tr_data)

# print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

In [9]:
predictions = model.predict(user_te, movie_te)

In [12]:
predictions_df = pd.DataFrame()
predictions_df['User'] = user_te
predictions_df['Movie'] = movie_te
predictions_df['Rating'] = predictions

predictions_df.head()

,User,Movie,Rating
0,44,1,4.013346
1,61,1,3.994558
2,67,1,3.406853
3,72,1,3.683364
4,86,1,3.744920


In [29]:
submission = create_csv_submission(predictions_df)

Creating submission file...


In [30]:
submission.to_csv("spotlight.csv")

In [7]:
# -----------Time: 0:11:05.077740, Loss: regression, n_iter: 50, l2: 1e-05, batch_size: 256, learning_rate: 0.0001, embedding_dim: 150, rmse: 0.9847699999809265-------------
model_en = ExplicitFactorizationModel(loss='regression',
                                   embedding_dim=150,  # latent dimensionality
                                   n_iter=50,  # number of epochs of training
                                   batch_size=256,  # minibatch size
                                   l2=1e-5,  # strength of L2 regularization
                                   learning_rate=0.0001,
                                   use_cuda=torch.cuda.is_available())

In [9]:
model_en.fit(train_data, verbose=True)

Epoch 0: loss 10.368889724238638
Epoch 1: loss 1.4260485237606093
Epoch 2: loss 1.0336816859483822
Epoch 3: loss 1.0059397320039067
Epoch 4: loss 1.0015464652720198
Epoch 5: loss 1.0004634259705338
Epoch 6: loss 0.9999525780623869
Epoch 7: loss 0.9994931217897141
Epoch 8: loss 0.9990492399462725
Epoch 9: loss 0.998692056338338
Epoch 10: loss 0.9980171905257692
Epoch 11: loss 0.9974114759790736
Epoch 12: loss 0.9965395889965645
Epoch 13: loss 0.9954941436330771
Epoch 14: loss 0.9940085963561982
Epoch 15: loss 0.992498328111233
Epoch 16: loss 0.990378863659046
Epoch 17: loss 0.9883615037263502
Epoch 18: loss 0.9860533290891452
Epoch 19: loss 0.9835981001875721
Epoch 20: loss 0.9810621489152125
Epoch 21: loss 0.9784472995916104
Epoch 22: loss 0.9756516686933566
Epoch 23: loss 0.9728106635842649
Epoch 24: loss 0.9700713580512337
Epoch 25: loss 0.9670148671269676
Epoch 26: loss 0.9638251552456304
Epoch 27: loss 0.9605304157801325
Epoch 28: loss 0.9565944997913789
Epoch 29: loss 0.9523780095

In [37]:
predictions_en = model_en.predict(user_te, movie_te)
predictions_df_en = pd.DataFrame()
predictions_df_en['User'] = user_te
predictions_df_en['Movie'] = movie_te
predictions_df_en['Rating'] = predictions_en

print(predictions_df_en.head())

submission_en = create_csv_submission(predictions_df_en)


   User  Movie    Rating
0    37      1  3.239807
1    73      1  2.971795
2   156      1  3.782236
3   160      1  3.106833
4   248      1  3.446185
Creating submission file...


In [38]:
submission_en.to_csv("spotlight_enlarged.csv")

In [8]:
# -----------Time: 0:05:04.112898, Loss: regression, n_iter: 20, l2: 1.3848863713938746e-15, batch_size: 256, learning_rate: 0.00018738174228603868, embedding_dim: 200, rmse: 0.9899572134017944-------------
model2 = ExplicitFactorizationModel(loss='regression',
                                   embedding_dim=200,  # latent dimensionality
                                   n_iter=20,  # number of epochs of training
                                   batch_size=256,  # minibatch size
                                   l2=1.3848863713938746e-15,  # strength of L2 regularization
                                   learning_rate=0.00018738174228603868,
                                   use_cuda=torch.cuda.is_available())

In [10]:
model2.fit(train_data, verbose=True)

Epoch 0: loss 5.42496124735704
Epoch 1: loss 1.0275038062826143
Epoch 2: loss 1.0165661359128872
Epoch 3: loss 1.0152426608597107
Epoch 4: loss 1.0141553155050531
Epoch 5: loss 1.0125589095130596
Epoch 6: loss 1.0100998618048966
Epoch 7: loss 1.0063645007715063
Epoch 8: loss 1.0007831244403356
Epoch 9: loss 0.9935231192856573
Epoch 10: loss 0.9848079140962026
Epoch 11: loss 0.9756279875916883
Epoch 12: loss 0.9647848259344884
Epoch 13: loss 0.9515543302902713
Epoch 14: loss 0.9348213510727976
Epoch 15: loss 0.9142228018827883
Epoch 16: loss 0.8892097840131807
Epoch 17: loss 0.859128309856969
Epoch 18: loss 0.8237335480986185
Epoch 19: loss 0.7826983401857287


In [39]:
predictions_2 = model2.predict(user_te, movie_te)
predictions_df_2 = pd.DataFrame()
predictions_df_2['User'] = user_te
predictions_df_2['Movie'] = movie_te
predictions_df_2['Rating'] = predictions_2

print(predictions_df_2.head())

submission_2 = create_csv_submission(predictions_df_2)
submission_2.to_csv("spotlight_2.csv")

   User  Movie    Rating
0    37      1  3.307779
1    73      1  3.104012
2   156      1  3.828374
3   160      1  3.423509
4   248      1  3.575156
Creating submission file...


In [21]:
pyfm = pd.read_csv("pyFM.csv", index_col=0)

(1176952, 2)

In [32]:
pyfm.head()

,Id,Prediction
0,r37_c1,3
1,r73_c1,3
2,r156_c1,4
3,r160_c1,3
4,r248_c1,4


In [35]:
def create_csv_submission(predictions):
    """Create submission file """
    print("Creating submission file...")
    def round_(x):
        if x > 5:
            return 5
        elif x < 1:
            return 1
        else:
            return round(x)
     
    predictions['Id'] = predictions.apply(lambda x: 'r{}_c{}'.format(int(x.User), int(x.Movie)), axis=1)
    predictions['Prediction'] = predictions.Rating.apply(lambda x: round_(x))
    return predictions[['Id', 'Prediction']]